<a href="https://colab.research.google.com/github/alexk2206/tds_capstone/blob/Domi-DEV/tryout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install on colab
# downgrade accelerate to make it compatible with Trainer API, see https://github.com/huggingface/accelerate/releases
!pip install accelerate==0.27.2
# after 4.38.2, top_k_top_p_filtering is removed, see https://github.com/huggingface/trl/issues/6
!pip install -U transformers==4.38.2
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.2.1
    Uninstalling accelerate-1.2.1:
      Successfully uninstalled accelerate-1.2.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 72.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency co

In [ ]:
import json
import pandas as pd

In [ ]:
url = "https://raw.githubusercontent.com/alexk2206/tds_capstone/refs/heads/main/questionnaires/questionnaire1.json?token=GHSAT0AAAAAACY3NQSAVESXWHFGJM3MFGEWZ32PBRQ"
Q1 = pd.read_json(url)
print(Q1)


HTTPError: HTTP Error 404: Not Found

### Converting answer options of multi select
In this block of code we are trying to convert the options of the questions posed in the questionnaires to a format, so that every combination of options is one option of the question.
For more clarity, let us consider a question with possible answers 'a' and 'b'. In the converted version, the options of that multi select questions are [], ['a'], ['b'], ['a', 'b'].

In [2]:
import json
import urllib
from itertools import chain, combinations

questions = []

for i in range (1, 6):
    url = f"https://raw.githubusercontent.com/alexk2206/tds_capstone/refs/heads/main/questionnaires/questionnaire{i}.json"
    response = urllib.request.urlopen(url)
    q = json.loads(response.read())
    for question in q:
      if question['type'] != 'MULTI_SELECT':
        continue
      else:
        options_json = question['options']
        options_list = [option['option'] for option in options_json]
        powerset = chain.from_iterable(combinations(options_list, r) for r in range(len(options_list)+1))
        question['options'] = [', '.join(list(s)) for s in powerset]
    questions = questions + q






**To be modfied:** load questions into json in the github repo

In [3]:
questions


[{'id': 'aa2d8cdd-0758-4035-b0b6-ca18e2f380d8',
  'type': 'SINGLE_SELECT',
  'question': 'Data processing consent',
  'options': [{'id': 'ee0437c0-6335-4b88-8bc5-d4eb8e2c68bf', 'option': 'Yes'},
   {'id': 'd357ab84-929f-440a-b9ad-42ff36402a53', 'option': 'No'}]},
 {'id': '12e1ed1d-edaa-4e93-8645-de3850e998f9',
  'type': 'SINGLE_SELECT',
  'question': 'Customer group',
  'options': [{'id': '53cc44fa-397b-40bf-8af8-3f629818c93b',
    'option': 'End User'},
   {'id': 'bab850d1-2de8-4cae-9f87-2c86d41615e8',
    'option': 'Wholesaler, Distributor'},
   {'id': '31ffae1e-f742-4a0b-b279-eb393483f075',
    'option': 'Consultant, Planner, Architect'},
   {'id': 'ed56f777-a7cd-48db-9d36-e2579c57e361', 'option': 'R&D'}]},
 {'id': '625012ae-9192-4cf6-a73d-55e1813d6014',
  'type': 'MULTI_SELECT',
  'question': 'Products interested in',
  'options': ['',
   'MY-SYSTEM',
   'Notion',
   'JTS',
   'JS EcoLine',
   'AKW100',
   'AX100',
   'MY-SYSTEM, Notion',
   'MY-SYSTEM, JTS',
   'MY-SYSTEM, JS EcoL

In [4]:
with open('q_changed_options.json', 'w') as fp:
    json.dump(questions, fp)

In [5]:
q_products_interested_in = [question for question in questions if question['id']== 'e95e34ce-1758-4037-9474-99d0557a7a29']
q_products_interested_in = q_products_interested_in[0]

q_products_interested_in['context'] = 'The contact person expressed interest in both Data Enrichment and Visit Report for their upcoming projects.'
q_products_interested_in['answer'] = "DataEnrichment, 'VisitReport"
q_products_interested_in['question'] = 'In which products was the contact person interested in?'

In [6]:
from transformers import AutoTokenizer, AutoModelForMultipleChoice
import torch

# Load the model and tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMultipleChoice.from_pretrained(model_name)


question = q_products_interested_in['question']
options = q_products_interested_in['options']
context = q_products_interested_in['context']

# Prepare inputs for multiple choice
inputs = tokenizer(
    [[context, question + " " + str(option)] for option in options],
    return_tensors="pt",
    padding=True,
    truncation=True
)

# Reshape inputs to match the expected format of the model
input_ids = inputs["input_ids"].reshape(1, len(options), -1)
attention_mask = inputs["attention_mask"].reshape(1, len(options), -1)

# Forward pass through the model
outputs = model(input_ids=input_ids, attention_mask=attention_mask)
logits = outputs.logits  # Shape: [batch_size, num_choices]

# Predict the option with the highest score
predicted_option = torch.argmax(logits, dim=1).item()

print(f"The predicted answer is: {options[predicted_option]}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The predicted answer is: BusinessCards, VisitReport, Data Cleansing


In [ ]:
# Print input IDs as strings
for idx, input_ids in enumerate(inputs["input_ids"]):
    print(f"Option {idx + 1}:")
    print(tokenizer.decode(input_ids, skip_special_tokens=True))
    print("-" * 50)

Option 1:
the contact person expressed interest in both data enrichment and visit report for their upcoming projects. in which products was the contact person interested in?
--------------------------------------------------
Option 2:
the contact person expressed interest in both data enrichment and visit report for their upcoming projects. in which products was the contact person interested in? businesscards
--------------------------------------------------
Option 3:
the contact person expressed interest in both data enrichment and visit report for their upcoming projects. in which products was the contact person interested in? dataenrichment
--------------------------------------------------
Option 4:
the contact person expressed interest in both data enrichment and visit report for their upcoming projects. in which products was the contact person interested in? visitreport
--------------------------------------------------
Option 5:
the contact person expressed interest in both dat

## Creating some neccesary functions

In [10]:
# check the accuracy of the huggingface model
from transformers import AutoTokenizer, AutoModelForMultipleChoice
import torch

# parameters:
# - model: dictionary of models for each question type
# - tokenizer:
# - questions: prepared QA-dataset
def accuracy(model, tokenizer, questions):
    correct_multi_select = 0
    correct_single_select = 0
    correct_text = 0
    correct_number = 0
    correct_date = 0
    correct_total = 0
    total = 0

    for question in questions:
        context = question['context']
        question_text = question['question']
        options = question['options']
        answer = question['answer']
        question_type = question['type']

        # Prepare inputs for multiple choice
        inputs = tokenizer(
            [[context, question_text + " " + str(option)] for option in options],
            return_tensors="pt",
            padding=True,
            truncation=True
        )
        # Reshape inputs to match the expected format of the model
        input_ids = inputs["input_ids"].reshape(1, len(options), -1)
        attention_mask = inputs["attention_mask"].reshape(1, len(options), -1)

        # Forward pass through the model
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits  # Shape: [batch_size, num_choices]

        # Predict the option with the highest score
        predicted_option = torch.argmax(logits, dim=1).item()
        if options[predicted_option] == answer:
            if question_type == 'MULTI_SELECT':
                correct_multi_select += 1
                total_multi_select += 1
            elif question_type == 'SINGLE_SELECT':
                correct_single_select += 1
                total_single_select += 1
            elif question_type == 'TEXT':
                correct_text += 1
                total_text += 1
            elif question_type == 'NUMBER':
                correct_number += 1
                total_number += 1
            elif question_type == 'DATE':
                correct_date += 1
                total_date += 1
            else:
              continue
            correct_total += 1
        total += 1
    accuracy_total = correct_total / total
    accuracy_multi_select = correct_multi_select / total_multi_select
    accuracy_single_select = correct_single_select / total_single_select
    accuracy_text = correct_text / total_text
    accuracy_number = correct_number / total_number
    accuracy_date = correct_date / total_date
    return accuracy_total, accuracy_multi_select, accuracy_single_select, accuracy_text, accuracy_number, accuracy_date

def output_model_quality(model, tokenizer, questions):
    accuracy_total, accuracy_multi_select, accuracy_single_select, accuracy_text, accuracy_number, accuracy_date = accuracy(model, tokenizer, questions)
    print(f"""Accuracy values of model: {model.name_or_path}\n
    Total: {accuracy_total}\n
    Multi-select: {accuracy_multi_select}\n
    Single-select: {accuracy_single_select}\n
    Text: {accuracy_text}\n
    Number: {accuracy_number}\n
    Date: {accuracy_date}\n""")
    return accuracy_total, accuracy_multi_select, accuracy_single_select, accuracy_text, accuracy_number, accuracy_date



In [11]:
# print model name of model
print(f"The model name: {model.name_or_path}")

The model name: bert-base-uncased
